In [1]:
from configparser import ConfigParser
import pandas as pd
import numpy as np
from pathlib import Path
import FuzzyMatch as fm
import DatafileIO as dio
import ComposeFunction as cf
import ConfigReader as cr
import DECAF

In [53]:
input_file = "tests/data/fuzzy_match_companies.xlsx"
inipath = "tests/config/processing/fuzzy_match_company.ini"
outname = "_OUT"
col1 = "reference_name"
col2 = "new_name"

config = cr.read_config(inipath)
all_fns = dio.read_functions_from_ini(inipath)
print(all_fns)

{'functions': ['FuzzyMatch.fuzzy_match_pairwise', 'FuzzyMatch.score_threshold'], 'cleaning': ['StringFormat.to_lower', 'StringFormat.trim_whitespace', 'StringFormat.strip_non_ascii', 'StringFormat.strip_punctuation', 'FuzzyMatch.remove_company_suffixes']}


In [55]:
# Setup - ini params
fns_proc = all_fns["functions"]
fns_clean = all_fns["cleaning"]
threshold_high = int(config["parameters"]["threshold_high"])
threshold_low = int(config["parameters"]["threshold_low"])
file_suffix = config["info"]["name"]

In [62]:
# Load & process data
data = dio.load_file(input_file)
data["clean1"] = cf.eval_functions_list(data[col1], fns_clean)
data["clean2"] = cf.eval_functions_list(data[col2], fns_clean)
# Fuzzy match
data["scores"] = fm.fuzzy_match_pairwise(data["clean1"], data["clean2"])
data["match"] = fm.score_threshold(data["scores"], threshold_high, threshold_low)

data

,reference_name,new_name,expected,thresh_expected,clean1,clean2,scores,match
0,Ballentine Partners LLC,"Ballentine Partners, LLC",match,1.0,ballentine partners,ballentine partners,100,True
1,Huron Capital Partners LLC,Huron Capital,match,1.0,huron capital partners,huron capital,90,True
2,USI Insurance Svc,Usi Insurance Services,match,1.0,usi insurance svc,usi insurance services,87,True
3,"New Heritage Capital, LLC",New Heritage Capital,match,1.0,new heritage capital,new heritage capital,100,True
4,Aureus,Aureus Capital,match,1.0,aureus,aureus capital,90,True
5,"Marsh USA, Inc.",Marsh Inc,match,1.0,marsh usa,marsh,90,True
6,Madison Capital LLC,Madison Capital Funding,match,1.0,madison capital,madison capital funding,90,True
7,Adams Street Partners LLC,"Partners, Adams St",match,1.0,adams street partners,partners adams st,86,True
8,Promus Bank & Trust,Promus Capital LLC,new,0.0,promus bank trust,promus capital,58,False
9,Huron Consulting Group,Huron Capital Partners LLC,new,0.0,huron consulting group,huron capital partners,50,False


In [40]:
DECAF.fuzzy_match_companies(
    input_file = "src/tests/data/fuzzy_match_companies.xlsx",
    col1 = "reference_name",
    col2 = "new_name"
)

TypeError: argument should be a str or an os.PathLike object where __fspath__ returns a str, not 'NoneType'